## Adding Technical Indicators

In [ ]:
import pandas as pd
import numpy as np
import time
from time import gmtime, strftime
import yfinance as yf
import ta
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from ta.momentum import StochasticOscillator, WilliamsRIndicator, TSIIndicator, ROCIndicator
from ta.trend import CCIIndicator
from ta.trend import MACD
from ta.momentum import TSIIndicator
from ta.volume import ChaikinMoneyFlowIndicator, VolumeWeightedAveragePrice
from ta.others import CumulativeReturnIndicator, DailyLogReturnIndicator
from ta.trend import SMAIndicator, EMAIndicator, MACD, ADXIndicator
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.volume import VolumeWeightedAveragePrice
from ta.others import DailyReturnIndicator, CumulativeReturnIndicator
from ta.trend import PSARIndicator

print("Started at " + str(strftime("%H:%M:%S", gmtime())))

# Load the combined dataset with Nifty 50 stock data from one Excel file
nifty50_excel_file = 'nifty50_stock_data.xlsx'
xls = pd.ExcelFile(nifty50_excel_file)

# Create an Excel writer to save the data to a single file
excel_writer = pd.ExcelWriter('combined_stock_data_single_sheet.xlsx', engine='xlsxwriter')

# Loop through each sheet (stock) in the Excel file
for k, sheet_name in enumerate(xls.sheet_names):
    start = time.time()

    # Get firm data for the current stock
    try:
        df = pd.read_excel(nifty50_excel_file, sheet_name=sheet_name)


        # Feature Engineering: Create additional features if needed
        # In this example, we'll use the previous day's closing price as a feature
        df['Previous_Close'] = df['Close'].shift(1)

        # Calculate Simple Moving Averages (SMA)
        df['5SMA'] = SMAIndicator(df['Close'], window=5).sma_indicator()
        df['10SMA'] = SMAIndicator(df['Close'], window=10).sma_indicator()
        df['20SMA'] = SMAIndicator(df['Close'], window=20).sma_indicator()
        df['50SMA'] = SMAIndicator(df['Close'], window=50).sma_indicator()
        df['100SMA'] = SMAIndicator(df['Close'], window=100).sma_indicator()
        df['200SMA'] = SMAIndicator(df['Close'], window=200).sma_indicator()
         # Calculate Exponential Moving Averages (EMA)
        df['5EMA'] = EMAIndicator(df['Close'], window=5).ema_indicator()
        df['10EMA'] = EMAIndicator(df['Close'], window=10).ema_indicator()
        df['20EMA'] = EMAIndicator(df['Close'], window=20).ema_indicator()

        # Calculate MACD
        macd = MACD(df['Close'], window_slow=26, window_fast=12, window_sign=9)
        df['MACD'] = macd.macd()
        df['MACD_signal'] = macd.macd_signal()

        # Calculate RSI
        df['RSI'] = RSIIndicator(df['Close']).rsi()

        # Calculate Parabolic SAR (PSAR) indicator
        psar_indicator = PSARIndicator(high=df['High'], low=df['Low'], close=df['Close'])
        df['PSAR'] = psar_indicator.psar()

        # Calculate Detrended Price Oscillator (DPO)
        # df['DPO'] = ta.momentum.DPOIndicator(df['Close']).dpo()

        # Calculating Vortex Indicator
        vortex_indicator = ta.trend.VortexIndicator(df['High'], df['Low'], df['Close'], window=200)
        df['vortex_indicator'] = vortex_indicator.vortex_indicator_diff()

        # Calculate Bollinger Bands
        bollinger = BollingerBands(df['Close'], window=20, window_dev=2)
        df['Upper_Band'] = bollinger.bollinger_hband()
        df['Lower_Band'] = bollinger.bollinger_lband()

        # Calculate Average True Range (ATR)
        df['ATR5'] = AverageTrueRange(df['High'], df['Low'], df['Close'], window=5).average_true_range()
        df['ATR10'] = AverageTrueRange(df['High'], df['Low'], df['Close'], window=10).average_true_range()
        df['ATR20'] = AverageTrueRange(df['High'], df['Low'], df['Close'], window=20).average_true_range()
        df['ATR50'] = AverageTrueRange(df['High'], df['Low'], df['Close'], window=50).average_true_range()

        # Calculate Stochastic Oscillator (Slowk and Slowd)
        stoch = StochasticOscillator(df['High'], df['Low'], df['Close'])
        df['Stoch_Signal'] = stoch.stoch_signal()
        df['Stoch'] = stoch.stoch()

        # Calculate Williams %R (WR)
        df['WR'] = WilliamsRIndicator(df['High'], df['Low'], df['Close']).williams_r()

        # Calculate True Strength Index
        df['TSI'] = TSIIndicator(df['Close'], window_slow=200, window_fast=100, fillna=False).tsi()

        # Calculate Average Directional Index (ADX)
        df['ADX'] = ADXIndicator(df['High'], df['Low'], df['Close'], window=14).adx()

        # Calculate Volume Weighted Average Price (VWAP)
        df['VWAP'] = VolumeWeightedAveragePrice(df['High'], df['Low'], df['Close'], df['Volume'], window=14).volume_weighted_average_price()

        # Calculate Daily and Cumulative Returns
        df['Daily_Return'] = DailyReturnIndicator(df['Close']).daily_return()
        df['Cumulative_Return'] = CumulativeReturnIndicator(df['Close']).cumulative_return()

        # Calculate Rate of Change
        df['ROC5'] = ROCIndicator(df['Close'], window=5).roc()
        df['ROC10'] = ROCIndicator(df['Close'], window=10).roc()
        df['ROC20'] = ROCIndicator(df['Close'], window=20).roc()
        df['ROC50'] = ROCIndicator(df['Close'], window=50).roc()
        df['ROC100'] = ROCIndicator(df['Close'], window=100).roc()
        df['ROC200'] = ROCIndicator(df['Close'], window=200).roc()

        # Calculate Chaikin Money Flow (CMF)
        df['CMF'] = ChaikinMoneyFlowIndicator(df['High'], df['Low'], df['Close'], df['Volume']).chaikin_money_flow()

        # Calculate Daily Log Returns
        df['Daily_Log_Return'] = DailyLogReturnIndicator(df['Close']).daily_log_return()

        # Replace infinite values with NaN
        df = df.replace([np.inf, -np.inf], np.nan).dropna()

        df.to_excel(excel_writer, sheet_name=sheet_name, index=False)

        print(f"Data for {sheet_name} added to the single sheet Excel file.")

    except Exception as e:
        print(f"Error fetching data for {sheet_name}: {str(e)}")

# Save the Excel workbook with all data for different companies in a single sheet
excel_writer.save()
excel_writer.close()

print("Finished. Data saved to 'combined_stock_data_single_sheet.xlsx'")
